In [6]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import *
from gensim.models import Word2Vec,KeyedVectors
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from spellchecker import SpellChecker

In [7]:
df=pd.read_csv('train.csv')

In [8]:
df.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [9]:
cdf=df.copy()

In [10]:
cdf.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [11]:
cdf.shape

(7274, 3)

In [12]:
cdf.describe()

,tweet_id,sentiment
count,7274.000000,7274.000000
mean,4531.736871,1.299148
std,2617.858745,0.607829
min,2.000000,0.000000
25%,2261.500000,1.000000
50%,4530.500000,1.000000
75%,6796.750000,2.000000
max,9092.000000,3.000000


In [13]:
cdf['sentiment'].value_counts()

1    4311
2    2382
0     456
3     125
Name: sentiment, dtype: int64

In [14]:
texts = cdf['tweet'].tolist()

In [15]:
texts

['#sxswnui #sxsw #apple defining language of touch with different dialects becoming smaller',
 'Learning ab Google doodles! All doodles should be light, funny &amp; innovative, with exceptions for significant occasions. #GoogleDoodle #sxsw',
 'one of the most in-your-face ex. of stealing the show in yrs RT @mention &quot;At #SXSW, Apple schools the mkt experts&quot;  {link}',
 "This iPhone #SXSW app would b pretty awesome if it didn't crash every 10mins during extended browsing. #Fuckit #Illmakeitwork",
 'Line outside the Apple store in Austin waiting for the new iPad #SXSW  {link}',
 '#technews One lone dude awaits iPad 2 at Apple\x89Ûªs SXSW store {link} #Tech_News #Apple #iPad_2 #SXSW #tablets #tech',
 'SXSW Tips, Prince, NPR Videos, Toy Shopping With Zuckerberg.\r\n{link}  #sxsw  #ipad',
 'NU user RT @mention New #UberSocial for #iPhone now in the App Store includes UberGuide to #SXSW sponsored by #Mashable',
 'Free #SXSW sampler on iTunes {link} #FreeMusic',
 'I think I might go a

In [16]:
cdf_lower = cdf['tweet'].str.lower()

In [17]:
cdf_lower

0       #sxswnui #sxsw #apple defining language of tou...
1       learning ab google doodles! all doodles should...
2       one of the most in-your-face ex. of stealing t...
3       this iphone #sxsw app would b pretty awesome i...
4       line outside the apple store in austin waiting...
5       #technews one lone dude awaits ipad 2 at apple...
6       sxsw tips, prince, npr videos, toy shopping wi...
7       nu user rt @mention new #ubersocial for #iphon...
8          free #sxsw sampler on itunes {link} #freemusic
9       i think i might go all weekend without seeing ...
10      rt @mention official #sxsw app û÷sxsw goûª b...
11      it's official! i'm buying an ipad. #sxsw #elevate
12      they're giving away ipad 2's, x boxes and book...
13      rt @mention we're officially at #sxsw! come by...
14      #companies to watch, from the #sxsw trade show...
15      rt @mention google marissa mayer, future of lo...
16      dl the #calyp app to get into #calyp casa at #...
17      well y

In [18]:
alphanumeric=['' for i in range(7274)]

for i in range(7274):
    a_string = cdf_lower[i]
    if type(a_string)!=float:
        for character in a_string:
            if character.isalnum():
                alphanumeric[i] += character
            elif character ==" ":
                alphanumeric[i] +=character
alphanumeric

['sxswnui sxsw apple defining language of touch with different dialects becoming smaller',
 'learning ab google doodles all doodles should be light funny amp innovative with exceptions for significant occasions googledoodle sxsw',
 'one of the most inyourface ex of stealing the show in yrs rt mention quotat sxsw apple schools the mkt expertsquot  link',
 'this iphone sxsw app would b pretty awesome if it didnt crash every 10mins during extended browsing fuckit illmakeitwork',
 'line outside the apple store in austin waiting for the new ipad sxsw  link',
 'technews one lone dude awaits ipad 2 at appleûªs sxsw store link technews apple ipad2 sxsw tablets tech',
 'sxsw tips prince npr videos toy shopping with zuckerberglink  sxsw  ipad',
 'nu user rt mention new ubersocial for iphone now in the app store includes uberguide to sxsw sponsored by mashable',
 'free sxsw sampler on itunes link freemusic',
 'i think i might go all weekend without seeing the same ipad case twice sxsw',
 'rt ment

In [19]:
type(alphanumeric)

list

In [20]:
tokenized_sents = [word_tokenize(i) for i in alphanumeric]
tokenized_sents

[['sxswnui',
  'sxsw',
  'apple',
  'defining',
  'language',
  'of',
  'touch',
  'with',
  'different',
  'dialects',
  'becoming',
  'smaller'],
 ['learning',
  'ab',
  'google',
  'doodles',
  'all',
  'doodles',
  'should',
  'be',
  'light',
  'funny',
  'amp',
  'innovative',
  'with',
  'exceptions',
  'for',
  'significant',
  'occasions',
  'googledoodle',
  'sxsw'],
 ['one',
  'of',
  'the',
  'most',
  'inyourface',
  'ex',
  'of',
  'stealing',
  'the',
  'show',
  'in',
  'yrs',
  'rt',
  'mention',
  'quotat',
  'sxsw',
  'apple',
  'schools',
  'the',
  'mkt',
  'expertsquot',
  'link'],
 ['this',
  'iphone',
  'sxsw',
  'app',
  'would',
  'b',
  'pretty',
  'awesome',
  'if',
  'it',
  'didnt',
  'crash',
  'every',
  '10mins',
  'during',
  'extended',
  'browsing',
  'fuckit',
  'illmakeitwork'],
 ['line',
  'outside',
  'the',
  'apple',
  'store',
  'in',
  'austin',
  'waiting',
  'for',
  'the',
  'new',
  'ipad',
  'sxsw',
  'link'],
 ['technews',
  'one',
  'l

In [21]:
stop_words = set(stopwords.words('english')) 

In [22]:
type(tokenized_sents)

list

In [23]:
# Use list comprehension to convert a list of lists to a flat list 
flatList = [ item for elem in tokenized_sents for item in elem]
print('Flat List : ', flatList)         

Flat List :  ['sxswnui', 'sxsw', 'apple', 'defining', 'language', 'of', 'touch', 'with', 'different', 'dialects', 'becoming', 'smaller', 'learning', 'ab', 'google', 'doodles', 'all', 'doodles', 'should', 'be', 'light', 'funny', 'amp', 'innovative', 'with', 'exceptions', 'for', 'significant', 'occasions', 'googledoodle', 'sxsw', 'one', 'of', 'the', 'most', 'inyourface', 'ex', 'of', 'stealing', 'the', 'show', 'in', 'yrs', 'rt', 'mention', 'quotat', 'sxsw', 'apple', 'schools', 'the', 'mkt', 'expertsquot', 'link', 'this', 'iphone', 'sxsw', 'app', 'would', 'b', 'pretty', 'awesome', 'if', 'it', 'didnt', 'crash', 'every', '10mins', 'during', 'extended', 'browsing', 'fuckit', 'illmakeitwork', 'line', 'outside', 'the', 'apple', 'store', 'in', 'austin', 'waiting', 'for', 'the', 'new', 'ipad', 'sxsw', 'link', 'technews', 'one', 'lone', 'dude', 'awaits', 'ipad', '2', 'at', 'appleûªs', 'sxsw', 'store', 'link', 'technews', 'apple', 'ipad2', 'sxsw', 'tablets', 'tech', 'sxsw', 'tips', 'prince', 'npr',

In [24]:
filtered_sentence = [w for w in flatList if not w in stop_words]
print(filtered_sentence)

['sxswnui', 'sxsw', 'apple', 'defining', 'language', 'touch', 'different', 'dialects', 'becoming', 'smaller', 'learning', 'ab', 'google', 'doodles', 'doodles', 'light', 'funny', 'amp', 'innovative', 'exceptions', 'significant', 'occasions', 'googledoodle', 'sxsw', 'one', 'inyourface', 'ex', 'stealing', 'show', 'yrs', 'rt', 'mention', 'quotat', 'sxsw', 'apple', 'schools', 'mkt', 'expertsquot', 'link', 'iphone', 'sxsw', 'app', 'would', 'b', 'pretty', 'awesome', 'didnt', 'crash', 'every', '10mins', 'extended', 'browsing', 'fuckit', 'illmakeitwork', 'line', 'outside', 'apple', 'store', 'austin', 'waiting', 'new', 'ipad', 'sxsw', 'link', 'technews', 'one', 'lone', 'dude', 'awaits', 'ipad', '2', 'appleûªs', 'sxsw', 'store', 'link', 'technews', 'apple', 'ipad2', 'sxsw', 'tablets', 'tech', 'sxsw', 'tips', 'prince', 'npr', 'videos', 'toy', 'shopping', 'zuckerberglink', 'sxsw', 'ipad', 'nu', 'user', 'rt', 'mention', 'new', 'ubersocial', 'iphone', 'app', 'store', 'includes', 'uberguide', 'sxsw', 

In [25]:
len(filtered_sentence)

89723

In [26]:
# tokens is a list of all tokens in corpus
freq_dist = nltk.FreqDist(filtered_sentence)
freq_dist


FreqDist({'sxsw': 7508, 'mention': 5512, 'link': 3380, 'rt': 2344, 'ipad': 1911, 'google': 1860, 'apple': 1727, 'iphone': 1214, 'store': 1188, 'new': 862, ...})

In [27]:
final_list = list(set(filtered_sentence))
final_list
    

['rolls',
 'nationwide',
 'adwords',
 'tm',
 'lanyrd',
 'snubor',
 'rileys',
 'managementmeasurement',
 'pickupline',
 'marissagoogle',
 '1230',
 'um',
 'innotribe',
 'discoveryû',
 '506',
 'suggestionskind',
 'quotgoogle',
 'contentquot',
 'scrape',
 'unequipped',
 'stacked',
 'become',
 'wp7',
 'innovating',
 'hackingwork',
 'hotsheet',
 'broke',
 'lbswars',
 'nadja',
 'roadie',
 'notsmell',
 'lags',
 '313',
 'sound',
 'spokewoman',
 'ducked',
 'industry',
 'postpc',
 'skeumorpism',
 '141st',
 'contentrules',
 'thatûªs',
 'instaprint',
 'begging',
 'reader',
 'withme',
 'slips',
 '20',
 'sheraton',
 'original',
 'ispyart',
 'autumn',
 'googleplaces',
 'concerts',
 'biomimicry',
 'foods',
 'fret',
 'transitapps',
 '¼ûäü',
 'axsure',
 'wings',
 'repressed',
 '031511',
 'snooping',
 'temperatures',
 'compilation',
 'busiest',
 'covering',
 'scary',
 'ringo',
 'black',
 'weasel',
 '10mins',
 'consequences',
 'aicn',
 'doh',
 'hardearned',
 '360idev',
 'nonprofit',
 'examples',
 'afteward

In [28]:
len(final_list)

9870

In [24]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(final_list)

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

wwwranaco
{'wwwranaco'}
owllove
{'owllove'}
larry
{'larry'}
windowsphone
{'windowsphone'}
pickmeupanipad2
{'pickmeupanipad2'}
comer
{'cometary', 'comer'}
galaxys4g
{'galaxys4g'}
setbacks
{'setbacks'}
ipad2û
{'ipad2û'}
normalpeople
{'normalpeople'}
quotfuture
{'quotfuture'}
ipadonly
{'ipadonly'}
networkingquot
{'networkingquot'}
1100am
{'1100am'}
propping
{'propping'}
citygoround
{'citygoround'}
caveats
{'caveats'}
hackingwork
{'hackingwork'}
kawasaki
{'kawasaki'}
sold-out
{'sold-out'}
meet
{'eetpu', 'setup', 'meetin', 'meets', 'meet', 'meatus'}
sxswsounds
{'sxswsounds'}
msusxsw
{'msusxsw'}
1045am3
{'1045am3'}
sash
{'sssh', 'sash', 'sesh', 'ssh', 'sshh'}
the
{'thus', 'bhs', 'rhs', "th'", 'thf', 'lhs', 'tbs', 'dhs', 'tho', 'thy', "t's", 'tes', 'tss', 'chs', 'thi', 'thos', 'ahs', 'tis', 'tls', 'tps', 'tvs', 'ehs', 'thu', 'this', 'tcs', 'thes', 'hs', 'tds', 'the', 'tas', 'thr', 'whs', 'ghs', 'th', 'vhs', 'tha', 'nhs', 'ts'}
nfcrfid
{'nfcrfid'}
quickcontact
{'quickcontact'}
coudbeeasier
{'c

marquess
{'marques', 'masques', 'marquess'}
a2b
{'a2b'}
11ntc
{'11ntc'}
view
{'viewer', 'views', 'viewed', 'view'}
pepsicostage
{'pepsicostage'}
interactivefilm
{'interactivefilm'}
spazmatics
{'spazmatics'}
burden
{'busmen', 'burden', 'bussed', 'bushes', 'busier', 'busied', 'gusev', 'bude', 'budded', 'bushed', 'buses', 'bused', 'buster', 'busses', 'busted', 'bushey', 'bushel', 'busies', 'husrev', 'busker'}
hashtags
{'hashtags'}
headachesquot
{'headachesquot'}
800115959p
{'800115959p'}
ûïspecialsû
{'ûïspecialsû'}
banksmoney
{'banksmoney'}
first-generation
{'first-generation'}
washable
{'washable'}
quotour
{'quotour'}
mindtouchers
{'mindtouchers'}
latinasintechû
{'latinasintechû'}
tech
{'techs', 'tech'}
mosaic
{'mosaics', 'mosaic'}
webs
{'webbs', 'webs'}
nonmacbook
{'nonmacbook'}
actionquot
{'actionquot'}
contentquot
{'contentquot'}
wlr
{'l/r', 'lcr', 'lar', 'wwr', 'lur', 'wlr', 'lnwr', 'lwp', 'nwr', 'ldr', 'lor', 'gwr', 'lwb', 'ltr', 'pwr', 'lwt', 'lsr', 'lr', 'wr', 'l-r', 'lw'}
version

ipaddesignheadaches
{'ipaddesignheadaches'}
quotgroupon
{'quotgroupon'}
fantastic
{'fantastic'}
binggoogle
{'binggoogle'}
miller
{'duller', 'driller', 'deller', 'miller', 'hiller', 'giller', 'killer', 'filler', 'tiller', 'ziller', 'dilley', 'piller'}
4squarequot
{'4squarequot'}
southbysouthwest
{'southbysouthwest'}
wwwskylinesnet
{'wwwskylinesnet'}
quotindustry
{'quotindustry'}
quotsteve
{'quotsteve'}
bassquot
{'bassquot'}
photos
{'photons', 'photos'}
iamagameshater
{'iamagameshater'}
allowingyou
{'allowingyou'}
donor
{'donat', 'donjon', 'conlon', 'donlan', 'donson', 'donor', "don't", 'donato', 'dont', 'dollop', 'dunlop', 'dilnot'}
mention
{'mention', 'mentions'}
square
{'square'}
quoted
{'quote', 'quoted', 'quotes', 'quoth', 'southey'}
screen
{'sheen', 'screen', 'schein', 'scheer'}
taptuquotapple
{'taptuquotapple'}
neediest
{'neediest'}
thewildernessdowntowncom
{'thewildernessdowntowncom'}
hardcore
{'hardcore'}
geekspringbreak
{'geekspringbreak'}
sussex
{'sussex'}
webbrowsing
{'webbro

likesquot
{'likesquot'}
tableware
{'tableware'}
missed
{'dished', 'kissed', 'disused', 'pissed', 'hissed', 'missed'}
stops
{'pitons', 'hips', 'tops', 'hits', 'hilltops', 'hops', 'hoops', 'hippos', 'stops'}
rulesquot
{'rulesquot'}
sxswsurvival
{'sxswsurvival'}
latham
{'latham', 'lata'}
webster
{'webster', 'pubmaster'}
n3
{'n=7', 'n3', 's37', 'g37', 'n7'}
catalogued
{'catalogued'}
sxswdad
{'sxswdad'}
quotmention
{'quotmention'}
salinas
{'salinas'}
mediamarketing
{'mediamarketing'}
brushstrokes
{'brushstrokes'}
internet
{'internet', 'internees'}
box
{'box', 'x-box'}
worklifeprogress
{'worklifeprogress'}
tcrncheb5fjs
{'tcrncheb5fjs'}
to
{'ton', 'toe', 'sto', 'ctos', 'ito', 'toa', 't5', 'pto', 'tow', 'tog', 'stow', 'atom', 'eton', 'to', 'stok', 'oto', 'mtoe', 'top', 'stop', 'ltom', 'tor', 'stoy', 'tot', 'atop', 'tod', 'btoe', 'tof', 'toc', 'too', 'stob', 'toi', 'tox', 'wto', 'vo5', 'tok', 'tol', 'bto', 'tom', 'itoh', 'rto', 'toy'}
apply
{'appn', 'appen', 'a11', 'ap1', 'apply', 'appy', 'appr

imrie
{'imrie'}
flashspecial
{'flashspecial'}
following
{'fulling', 'following'}
longford
{'long-form', 'longford'}
kiiiiiilling
{'kiiiiiilling'}
socialnews
{'socialnews'}
thisisdare
{'thisisdare'}
anvil
{'anvil', 'adil'}
pageant
{'palest', 'pageant'}
rowdy
{'howey', 'rowdy', 'dowdy'}
musicviz
{'musicviz'}
quotmistakes
{'quotmistakes'}
catphysics
{'catphysics'}
ajaxquot
{'ajaxquot'}
restraints
{'restraints'}
awkward
{'awkward', 'backward'}
popuitp
{'popuitp'}
teletubbie
{'teletubbie'}
windows
{'windows'}
youse
{'youse'}
essdub
{'essdub'}
grandad
{'grandad'}
lucy
{'lucy', 'lucas'}
quoits
{'quoits'}
quotpartyingquot
{'quotpartyingquot'}
imthetype
{'imthetype'}
11quot
{'11quot'}
sxsw500
{'sxsw500'}
rubens
{'rubens', 'reuben'}
nadia
{'nadia', 'nada'}
ipadformatted
{'ipadformatted'}
eight-bit
{'eight-bit'}
therocksreport
{'therocksreport'}
quotbetter
{'quotbetter'}
bigmistake
{'bigmistake'}
cliff
{'cli', 'cliff', 'clef', 'coif', 'clio', 'clift', 'clip', 'clit'}
arrive
{'arrive', 'arrigo'}
u

...

quottheyre
{'quottheyre'}
enoughquot
{'enoughquot'}
fightthepaddle
{'fightthepaddle'}
macworld
{'macworld'}
a
{'d', 'o', 't', 'x', 'l', 'a', 'r', 'v', 'q', 's', 'n', 'm', 'h', 'c', 'b', 'p', 'f', 'g', 'j', 'i', 'z', 'y', 'k', 'u', 'e', 'w'}
happensxsw
{'happensxsw'}
at
{'ax', 'at1', 'ata', 'tx', 'atc', 'aix', 'ttx', 'atb', 'gtx', 'att', 'atn', 'ftx', 'atm', 'at3', 'atg', 'a-x', 'tax', 'ati', 'a3x', 'atv', 'at2', 'at', 'ate', 'aux', 'at.', 'atp', 'ats', 'ath', 'atk'}
agent
{'aunt', 'quest', 'ascent', 'quett', 'sequent', 'aquino', 'ardent', 'squint', 'advent', 'agent', 'absent', 'fluent', 'quint', 'accent', 'quant', 'assent', 'argent'}
latium
{'latium'}
swesxsw
{'swesxsw'}
schedulestheir
{'schedulestheir'}
veto
{'evo', 'veto', 'vero', 'vivo'}
lover
{'lowther', 'lover', 'loehr', 'lovelier'}
eventprofs
{'eventprofs'}
powerglove
{'powerglove'}
twitternext
{'twitternext'}
quotdrink
{'quotdrink'}
quatrains
{'quatrains'}
noah
{'woh', 'noah', 'woad', 'woan', 'wah'}
livesteam
{'livesteam'}
macsq

jordanhill
{'jordanhill'}
geekfest
{'geekfest'}
ihearnetwork
{'ihearnetwork'}
concert-goers
{'concert-goers'}
likes
{'likes', 'pikes', 'bikes', 'mikes', 'dikes', 'hikes', 'sikes', 'yokes'}
row
{'ww', 'rmw', 'rw', 'tww', 'raw', 'nww', 'aww', 'rwc', 'row'}
dizzy
{'tizzy', 'dizzy', 'buzzy', 'izzy', 'lizzy', 'fizzy'}
storymeaning
{'storymeaning'}
are
{'ara', 'asw', 'aru', 'arm', 'arf', 'ars', 'arc', 'drw', 'alw', 'aw', 'rw', 'aww', 'a.w', 'arp', 'are', 'ar', 'ark', 'ari', 'arg', 'art', 'arr', 'trw', 'raw'}
successful
{'successful'}
miriam
{'miriam', 'irian', 'priam'}
wwwsocialmediabumtumblrcom
{'wwwsocialmediabumtumblrcom'}
downtownquot
{'downtownquot'}
bestappever
{'bestappever'}
outcast
{'outcast', 'outlast'}
day
{'day'}
c2
{'c2s', 'c2', 'c=3', 'p23', 'cd3', 'cm3', 'a23', 'm23', 's23', 'c3'}
context
{'context'}
in case
{'encase', 'incas', 'in case'}
worldquot
{'worldquot'}
quotsocial
{'quotsocial'}
of
{'n4', 'd3', 'ic', 'rb', 's2', "a'", 'ps', 'ua', 'yz', 'nk', 'c8', 'ol', 'iu', 'wn', 'c

davis
{'satis', 'smuts', 'sais', 'elvis', 'devis', 'hmis', 'nevis', 'svqs', 'amis', 'smit', 'mavis', 'solis', 'vis', 'mis', 'levis', 'smail', 'avis', 'savin', 'mais', 'bovis', 'hovis', 'mvs', 'sms', 'semis', 'sufis', 'divis', 'bevis', 'saris', 'spivs', 'davis', 'sssis', 'pmis', 'skis', 'smrs', 'sis', 'smogs', 'wyvis', 'sims', 'saves', 'suis', 'smes'}
geosocial
{'geosocial'}
relocation
{'relocation', 're-location'}
padlock
{'padlock'}
tweetdeck
{'tweetdeck'}
showusyouricrazy
{'showusyouricrazy'}
webster
{'webster', 'websters'}
inyourface
{'inyourface'}
arab
{'grab', 'ahab', 'dab', 'moab', 'drab', 'tab', 'chab', 'saab', 'mab', 'ccab', 'nab', 'cab', 'fab', 'jab', 'stab', 'bab', 'lab', 'rab', 'crab', 'flab', 'e2a', 'arab', 'slab', 'zab', 'swab', 'a2b', 'ab', 'raab', 'gab', 'oab', 'scab'}
quotcheck
{'quotcheck'}
tsunamithe
{'tsunamithe'}
ms
{'ams', 'kms', 'zxs', 'rms', 'ems', 'nms', 'zds', "z's", 'cms', 'vms', 'hms', 'fms', 'bms', 'sms', 'ums', 'dms', 'pms', 'zbs', 'gms', 'lms', 'ims', 'ms'

6thcongress
{'6thcongress'}
zazzlesxsw
{'zazzlesxsw'}
urthotscom
{'urthotscom'}
khan
{'kwan', 'khan', 'kean', 'xan', 'klan', 'kan', 'kuan'}
ipadperson
{'ipadperson'}
digitalblasphemy
{'digitalblasphemy'}
debbie
{'herbie', 'debbie', 'begbie', 'newe', 'nettie', 'kelbie', 'newi', 'ebbie', 'nesbit', 'nessie', 'newbon', 'nellie', 'newby', 'nebe'}
cover
{'cov', 'cover', 'cour', 'cove', 'cvr', 'coir', 'corr', 'cor'}
quotmapping
{'quotmapping'}
typing
{'hyping', 'typing'}
linear
{'lanier', 'linear', 'loncar', 'lancer', 'lunar', 'langer', 'lanka', 'langur', 'anwar', 'lantau', 'landau', 'lascar', 'lanark', 'lantra', 'lana', 'lazar', 'hangar', 'lanham', 'landor', 'ladhar', 'lander', 'lanyard', 'lamar', 'bandar', 'planar', 'lannah', 'langan', 'lankan', 'mannar'}
wowltltûïmention
{'wowltltûïmention'}
dayearthquake
{'dayearthquake'}
activation
{'activation'}
jambon
{'jambon'}
bitlyajs2011
{'bitlyajs2011'}
pm
{'udpm', 'rpm', 'spm', 'capm', 'pm', 'tpm', 'ipm', 'cpm', 'mpm', 'ppm', 'ccpm', 'wpm', 'bpm'

laurieshook
{'laurieshook'}
mueller
{'beller', 'mueller', 'buller'}
techsavvy
{'techsavvy'}
corporatewhore
{'corporatewhore'}
mobilesearch
{'mobilesearch'}
catfightquot
{'catfightquot'}
compress
{'oppress', 'compress'}
oooorkut
{'oooorkut'}
clevelandsteamer
{'clevelandsteamer'}
thosequot
{'thosequot'}
weekend
{'weekend'}
httptco26svo3m
{'httptco26svo3m'}
chzbrgr
{'chzbrgr'}
jade
{'nadi', 'madi', 'jedi', 'jade', 'hadi', 'judi', 'jodi', 'jaci', 'kadi', 'padi', 'jai', 'adi', 'wadi'}
betterquot
{'betterquot'}
richdeep
{'richdeep'}
bettercloud
{'bettercloud'}
hiringquot
{'hiringquot'}
truestory
{'truestory'}
wikimedia
{'wikimedia'}
iphoneapp
{'iphoneapp'}
successquotmention
{'successquotmention'}
heart-rate
{'heart-rate'}
offside
{'offside', 'off-site'}
real-estate
{'real-estate'}
outright
{'outright'}
obe
{'ob', 'obs', 'mbv', 'dbv', 'ov', 'obe', 'bv', 'hbv', 'opv'}
and
{'asp', 'ads', 'asw', 'hsd', 'bsd', 'csd', 'asda', 'psd', 'asi', 'as', 'ask', 'sd', 'a-d', 'asm', 'aid', 'asv', 'add', 'ss

Lemmatization

In [33]:
#initialise word lematizer
lemmatizer = WordNetLemmatizer()

In [40]:
for i in range(9870):
    print(lemmatizer.lemmatize(final_list[i]))
    
    


roll
nationwide
adwords
tm
lanyrd
snubor
riley
managementmeasurement
pickupline
marissagoogle
1230
um
innotribe
discoveryû
506
suggestionskind
quotgoogle
contentquot
scrape
unequipped
stacked
become
wp7
innovating
hackingwork
hotsheet
broke
lbswars
nadja
roadie
notsmell
lag
313
sound
spokewoman
ducked
industry
postpc
skeumorpism
141st
contentrules
thatûªs
instaprint
begging
reader
withme
slip
20
sheraton
original
ispyart
autumn
googleplaces
concert
biomimicry
food
fret
transitapps
¼ûäü
axsure
wing
repressed
031511
snooping
temperature
compilation
busiest
covering
scary
ringo
black
weasel
10mins
consequence
aicn
doh
hardearned
360idev
nonprofit
example
afteward
pgi
yummy
comparison
brain
magnet
hamsandwich
increase
checking
biz
bday
dorkinout
cutts
tablet
ditto
ho
backpack
yield
hcinno
psych
concierge
shea
smarter
referral
webinar
droppin
contextclues
post
wr
quotcameraquot
retail
relay
splash
keep
nifty
coveting
dtas
quotcirclequot
pat
foodspottings
moment
fin
wisconsin
deck
dear
pract

hollow
mmod
homogeneity
dubbing
invades
originally
tmr
eventprofs
sundance
pushio
ship
acoustic
gr2l2
please
japanquot
timed
kthxbai
able
foot
2moro
ensue
pretty
wonder
sbux
fbtw
archive
in
dirty
031411
preference
followed
login
quotbizzyquot
slap
category
hauling
sponsoû
craziness
hopefully
appointment
plan
select
consumerization
weeping
corner
hungry
quotnostalgicquot
quotbig
woo
sandwich
ltanxiety
base
sxswnice
though
strange
continuous
reassured
demonstrate
used
teach
tweetcaster
latinasintech
discoverablequot
motherboard
combine
48
bound
txt
iangogame
run
winamp
maar
quotipodquot
craigslist
eat
779
4chans
spot
pissing
suckquot
matching
17
roomquot
lining
mantra
lthere
picking
hangover3
conveniently
mylunch
temporary
tamale
pavement
oki
hoax
alex
disturbing
look
beevil
kiip
dd
childish
grrr
c43
direction
uncertainty
feed
apptrophy
joint
latitude
amble
spectacle
roaming
realtime
newandroid
overnight
transparently
frame
heading
torturous
notsomuch
ipadwinning
wmy
african
snuggie
wooh

shatter
tomato
oooh
outbrain
ppl
revolutionary
2430ready
delight
may
nokiaconnects
error
test
disaster
djroe
everywherequot
divorcedquot
h264
connected
spending
xtra
n25
wave
breath
spontaniety
n5
geo
apartment
beat
huh
casually
quotvintagequot
sim
heaving
tcrnchfcs45j
answer
reacquainted
netbook
keynote
youareinteractive
opener
photovideostatus
purchase
seth
4sqwill
galaxys4g
doctor
ago
quotvia
smudgy
red
bbq
private
standard
g2
ooc
avoid
worldwide
sxxpress
73
perimeter
filmasterûªs
exclusive
sxswthe
rigeur
atm
addiction
jalen
badger
alguien
michael
rate
tattoo
fine
webpronews
conferencesquot
consumer
n26
experimenting
problem
kick
entertain
idealab
here
tight
click
ajax
halftone
u
hooray
charge
hum
unofficial
pubcamp
quoted
closer
service
paywall
designerdrugs
äûâüââãªãâäâäâäçáîùã¾ûâãôûârt
retreat
wan
tammi
follower
myopia
tempted
admitting
100
foursquaregowalla
reach
launch
smscifi
una
15k
valuable
quotcomparing
crash
choplifter
authenticationdesign
rancewilemon
uxamandroid
warp
red

quotables
ritas
swonderlin
doo
funnysad
powerhouse
flav
140608
zzzs
quotsoonquot
gpsaware
inû
cool
want
p
intending
gut
superbia
girl
fatigue
full
discover
piece
jamin
avoiding
including
engineering
spencer
prop
tcrncheb5fjs
peak
ª¼
unstable
bait
observer
bad
sxswsa
choiceû
earphone
tom
dance
0610
h4ackers
acerbic
assistivetech
geekfest
detailed
baller
electric
backup
bug
fastsociety
seems
integrated
intel
tinkering
beating
register
googleaclueff
imanoutcast
racing
bluetooth
documented
visitorsû
thenextweb
quoto
beat
klick
rt
lifter
attractive
coolas
pod
fool
gadgetenvy
imthetype
flavor
travoltas
colin
deliciousness
saber
top
mocking
arabspring
quotoptionsquot
ireports
marrisa
zimride
3am
browserwars
quotipad
freecreditscore
hotel
overlaid
829
loud
damon
explorer
x
detached
nearly
citymarket
syncd
eecms
class
owning
girlcrush
trinity
surprising
debut
augcomm
startup
sandwichawesome
guru
havent
quotflash
720
applestore
quotcab4mequot
bull
holytrafficjams
ferris
tobeannounced
fact
box
of

hisher
quotwill
waffling
gofigure
redboxgilt
95
tee
ltlt
likened
compact
ackward
38
o
divide
showcased
sm2
bright
insightful
agoquot
seemingly
principle
demoed
people
tt
horizon
aclugoogle
319
stopped
slew
untrue
split
emc
ie6
plush
eloquently
quotaustin
reprieve
3d
surface
chilling
homie
surely
quotgot
authenticator
struggle
64g
tues
ouchhisshot
playstation
finger
wiiings
burner
info
peaked
nutshell
thurs
music
client
reconcile
1amp2
lift
rating
updating
hockeyfan
getjarsxsw
redcross
quotmy
crashing
trend
journalism
pressie
crossed
reunite
rush
teamandroid
headphone
goggles
teathering
spiltbeer
effectively
identity
betainvites
steam
awaits
micro
chief
edtech
affiliated
golden
irony
chuck
simplicity
tabletsamazing
book
drop
bashing
entirely
future
jqtouch
coupon
makin
fuckin
another
247
11m
nba
charm
ranking
comcom
rose
eod
glass
spotting
wish
nasatweetup
servicestory
destruction
allowingyou
sxwsi
alone
setup
deck
designer
914
trace
tiff
sydney
smooth
virtualoffice
guilttrip
crowed
nex

scan
phil
thedaily
context
loser
reported
wwwsocialmediabumtumblrcom
flawless
sleepy
parsing
sxswers
smartypants
pubsubhubbub
expressed
paper
q
marker
find
conan
thousand
makingquot
value
quotapptrophyquot
lion
clearly
jackass
empowered
mustresistmactemptation
princess
wwwadloopzcom
priestess
render
bldg
umm
playsxswû
truly
available
uh
building
hysterical
think
messina
friday
prototype
autonomous
skull
drumroll
swapping
earthhour
actsofsharingcom
fulltime
pov
novelty
selfreferential
43
entrepreneur
thefutureismobile
crap
anything
ctr
buzzrt
recs
study
would
exposing
tester
phonecamera
newapplestoreaustin
thirsty
stereoquot
removeburn
globally
bot
atx
thunder
content
convinced
redirect
apartmovie
reconsidering
crapped
scoping
option
z9
bet
seta
conversationsû
yellow
v12
tablet
mapped
pumped
page
couple
usar
boomer
thatll
step
let
musedchat
tag
september
anyway
exgoogle
12ab
grouponlike
try
spy
pack
capitalist
growing
embarrassing
exhibit
john
genius
set
challenge
comms
el
shot
speaking

trigger
guest
tool
recharge
networkquot
brighteyes
yelpstyle
leo
yourselvesseems
deshi80227mention
autograph
beprepared
filterû
uifail
advil
ë
exploiting
digging
specimen
cut
acrosse
4chan4eva
cedar
itsnot
awhile
constant
frineds
serf
advance
applauds
tuh
knock
early
overall
trump
predictability
craigslistkiller
creature
gps
chattingtexting
dropping
vp
storm
yearsquot
mcrees
truth
cart
handsfree
former
mentioned
metric
statement
bizness
atåêsxsw
geekbeat
knitted
greatly
uncharged
research
added
interacting
coined
artikelen
aight
dispenser
ti
proprietary
badge
know
dancin
ridiculously
parenting
30th
applequot
dash
engagement
wonder
store
autism
tuneup
exceptionally
mean
couch
divorce
compliment
apac
travelled
visit
plane
pointlow
trying
googleûªs
shopping
bat
murphy
till
burning
rockaroke
showusyouricrazy
kneejockey
tonite
jquery
scavenger
schema
2g
b4
force
houston
longest
7000
playlist
rey
pitchfork
optimal
convention
type
cup
taker
drink
posterousevents
liver
practice
digisxsw
thatli

101
bart
sxswsurvival
script
disagree
noticed
eddy
enough
2rd
quotwhy
scrm
c
evacuation
poetry
searching
charlie
lady
happily
test
quotlost
quotfloorsittingquot
dotco
straight
portion
shoulder
electronics
contest
desk
connection
livestream
route
articulating
niceness
ranked
pen
tidbit
theinternetlink
ill
giant
musicconnex
turning
advising
garyvee
katrina
techiesamp
favorite
aka
booya
search
powermatteam
ûïfoursquare
w
monopoly
quotmerrierquot
bavcorgimpact
crossing
marshall
outlook
discovered
den
get
expected
periscope
skiing
transit
across
lisa
location
midem
handle
promo
task
historical
fuss
gaining
onsxsw
author
afar
bruise
smallbiz
ohh
ventured
motorola
local
miamibeach
nowsxsw
iphonebberryandroid
proof
warmth
eventseekr
hit
mediamarketing
guy
introduces
doofusness
threat
hannukah
atomic
overload
cash
hint
battlela
ny
window
pensei
vast
doen
circle
answr
intentionally
yur
heathcare
criticism
pot
honor
contact
webkit
brightness
toodamnlucky
doodle
face
doomed
repaid
imthatgood
previ

Stemming

In [37]:
ps=PorterStemmer()

In [41]:
for i in range(9870):
    print(ps.stem(final_list[i]))

roll
nationwid
adword
tm
lanyrd
snubor
riley
managementmeasur
pickuplin
marissagoogl
1230
um
innotrib
discoveryû
506
suggestionskind
quotgoogl
contentquot
scrape
unequip
stack
becom
wp7
innov
hackingwork
hotsheet
broke
lbswar
nadja
roadi
notsmel
lag
313
sound
spokewoman
duck
industri
postpc
skeumorp
141st
contentrul
thatûª
instaprint
beg
reader
withm
slip
20
sheraton
origin
ispyart
autumn
googleplac
concert
biomimicri
food
fret
transitapp
¼ûäü
axsur
wing
repress
031511
snoop
temperatur
compil
busiest
cover
scari
ringo
black
weasel
10min
consequ
aicn
doh
hardearn
360idev
nonprofit
exampl
afteward
pgi
yummi
comparison
brain
magnet
hamsandwich
increas
check
biz
bday
dorkinout
cutt
tablet
ditto
ho
backpack
yield
hcinno
psych
concierg
shea
smarter
referr
webinar
droppin
contextclu
post
wr
quotcameraquot
retail
relay
splash
keep
nifti
covet
dta
quotcirclequot
pat
foodspot
moment
fin
wisconsin
deck
dear
practic
internetonlinewebsitecom
webdoccom
imagin
scan
awww
idiot
abt
repair
2year
blow
pr

quotb
shawn
bestworstthingev
groupawar
18
lamewad
belgium
gener
14th
mophi
cell
window
reel
cleantech
easili
scream
iradar
atrix
llc
head
homeless
refer
discuss
hard
loui
upload
sadli
cisco
gorgeou
allow
everywherei
regular
grate
harbor
whatev
wot
featur
seevisit
amus
fastû
hail
cpa
socialgood
joepiee
phoenix
point
return
timberlak
yawn
stow
sxswchi
grown
forest
gtd
speakquot
leas
producto
sxsw500
tonight
wrong
stoke
luggag
1315
winner
develop
foråêment
beard
bang
httpswwwfacebookcomeventphpeid141164002609303
chip
aw
goggler
dudesjust
crave
waffl
socialsearch
recon
frankenstein
meyer
alarm
quotsxsw
cbustech
alpha
360
impart
diseas
ring
demo
renew
tournament
felix
texttospeech
attendong
stayingal
ericschura
trust
crazytim
consum
execut
larri
sxswa
decenc
planesairport
selfish
silent
palett
7th
dont
1986
payitforward
session
conflictquot
redlas
vuelta
guardian
bolinsxsw
mission
quotthank
star
amex
sneaki
constantcontact
behaviour
collect
wu
true
pseudoretweet
rental
incred
quotmay
promot

3bil
sander
facebookquot
arboretum
brazil
lesson
anonym
support
disavow
steal
marissaumar
pictur
affirm
812
wcaveat
rise
crunch
product
start
greaaat
latism
screenscrap
aclu
150mm
corrupt
polit
envi
failquot
droid
dosta
osmpw
bag
nonsxsw
quotscarquot
sn
ah
htxsw
worri
tummi
wedmon
1990style
iphoneipadio
pit
frm
thu
qualcosa
fr
ask
prioriti
boundari
superlucki
fantast
lowest
lug
prematur
breach
parti
winningquot
temptat
sxswnui
barri
deadli
curfew
lt3
row
groundlink
disk
kick
passì
chime
fess
spacedont
within
dash
optiscan
theyv
school
becam
whetherappl
experiment
hehe
80
screenquot
marcin
conscious
futuremf
loveh
delight
ona
dosent
woman
toy
chavez
bandwidth
lifetim
10000
tweethous
215
leather
quotright
µ
scienc
siren
chk
servic
funni
ther
yourmom
carpool
darl
speakersquot
mullet
äá
culinari
unveil
swsurrog
three
ehteam
akhirnya
manag
macpc
corner
tryna
blog
wacki
math
trusti
mysteri
record
em
lcd
syncaps
fightthepaddl
pair
restaurantsr
liberti
probar
swissmad
whimsic
fado
chng
girl
jo

critic
smentisc
collect
th
colour
100
ur
brawl
highlight
itouru
imag
serìá
fathom
5pm
911tweet
minut
btwn
casualeducationgam
age
pictur
walk
loung
tribe
shortli
sxswwill
goog
lookalik
condensedquot
holler
card
hope
fragment
fellowship
friggen
support
perman
rescu
nonmacbook
elitebook
bag
sangr
fogo
gestur
quotment
rob
ticket
hmmûa
uj
brighten
takeoverquot
studentsforcleanwaterorgrsvp
streetview
qrank
idioci
aquent
vestibul
note
quotwin
two
danger
input
abil
99cent
prepar
billmurray
quiet
muro
sdk
bobbi
appear
leanux
towel
retweet
quotbright
ran
esp
moor
flar
makeri
beach
bust
winelibrari
cmon
commerci
timeeveri
phenomen
distort
veriphon
v2
sxswnl
networkcircl
rockin
hand
605
existenti
bu
31111
writin
mpact
worklifeprogress
crosscheckin
potentialû
freespeech
chzbrgr
mover
scene
troubl
usabl
expert
almost
incub
barton
major
borrow
account
cobra
replac
attsxsw
11bil
fruit
render
solar
shame
glowstick
ipoo
mobile491
newest
webbrows
sxswdad
laurieshook
inform
show
corner
350
mister
portabl


deliv
interest
e
audienc
hackerspac
metaphor
school
program
ûï35
pornthi
clear
cloud
sxswh
economyquot
outquot
veteran
tweetup
vodka
swoon
crashingquot
sloanxsw
batphon
republish
point
multichannel
47
16162
naiv
lbseverywher
linkedin
theft
webbi
blue
allhat3
und
stimul
visual
pinbal
attempt
poopoo
time
univers
balsam
hollrbackcom
longer
flow
sf
contact
nc
piti
smmnextgen
speak
toolongform
roast
exist
drawn
guysget
sxswiû
economi
antiqu
analog
overs
nisxsw
epicquot
fight
memolan
glut
gamesalad
urthotscom
liveley
heat
techgeek
rock
howard
circlesrwwtof6bcet
pro
printisnotdead
n33
dell
ipad
citygoround
movement
global
anti
josh
reward
geogam
hide
walk
terrac
savesxsw
decad
quotsometh
mvideo
pal
incorrect
monday
market
demand
httpbitlyieavob
rim
chic
stat
delici
pure
toy
fastbal
quotwrongquot
irl
purchas
open
1045am3
exchang
workshop
registr
bose
set
slice
sxswapi
firstgener
high5
bequot
stripe
wverizon
valid
austin
video
overshadow
xperia
creator
airport
chain
rulesquot
eduvc
charg
quotmo

grouponlik
tri
spi
pack
capitalist
grow
embarrass
exhibit
john
geniu
set
challeng
comm
els
shot
speak
industrialdesign
rad
celebr
pakistan
cleveland
brows
shell
bro
fake
cheeki
quotridicul
jare
petit
intric
argu
hackathon
therefor
control
ink
involv
l
highfiv
pan
display
programm
networklink
miss
bing
autom
4thi
ep3
pixel
cusit
host
definit
qampa
jacinto
skype
quotiphon
sxprotect
show
2011quot
wguy
bug
420
interact
market
version
footag
suicid
largest
lack
dang
offer
condens
triall
luci
show
quotdead
smileyparti
i21
shuffleboard
well
appletogo
appsand
turn
cooper
launchquot
redeem
sxswcare
41
donlin
recoveri
clariti
box
marissamey
darker
vet
bsocial
wander
mhealth
hip
vid
cw
rt6n79xp37ff
iphoneipod
light
ûïspecialsû
rival
sneak
soul
badgeless
playhopskoch
pi
safe
jeopardi
irish
strike
abacu
pager
impress
pre
ridic
chìáo
wevent
anû
sxswbigbrand
virtual
artallaround
york
trustworthi
popupshop
gmail
sxfl
present
conservebatterylif
advisori
lotteri
mill
ive
vizthink
fancraz
seattl
sharegat

autism
tuneup
except
mean
couch
divorc
compliment
apac
travel
visit
plane
pointlow
tri
googleûª
shop
bat
murphi
till
burn
rockarok
showusyouricrazi
kneejockey
tonit
jqueri
scaveng
schema
2g
b4
forc
houston
longest
7000
playlist
rey
pitchfork
optim
convent
type
cup
taker
drink
posterousev
liver
practic
digisxsw
thatlink
junki
wifionli
rebecca
yen
gotten
khoi
step
crave
ao
space
aliv
voluntarili
onequot
destin
talib
confirm
quottheyr
taxi
sxswbound
park
deserv
polic
takeonefortheteam
softlay
stream
kind
iamagameshat
pig
watson
smm
clear
squar
kinda
thom
fond
hope
cri
akqa
950
reserv
stupid
courtsid
thecivilwar
sunday
rumor
nije
tt66h9tjn4ye
ringer
åð
vai
nfcrfid
2
hire
coolgoofi
cellular
hollrback
may
cast
opinion
prop
mention
preced
ipadi
collect
iqlab
mountain
8th
horror
wrote
expert
dock
windowsquot
citeren
inventori
nerd
panic
312
say
mayertak
quotbank
ipaddesign
googlez
handheld
govern
web
dog
highvolum
quothighqu
especi
ridicul
quotcloudquot
bump
consumerfac
lego
other
blinksal
oft

seenatsxsw2011
tragedi
bother
present
mentionm
onetoon
start
librari
kshepherd
kh
bizgym
hear
sever
chang
admiss
releas
pop
googlesit
bitlyea1zgd
lykwxhpyterh
quotha
corp
publish
cd
headsup
decid
evrmobil
honor
accordion
iriam
crack
quotther
error
220
fedex
zlf
scale
schemaquot
morningquot
improv
semi
roast
250
teas
altern
obviou
quotad
convor
celebr
meat
youneedthi
brick
popplet
317
pub
upstair
casepouch
imac
ya
gay
touch
trueûïment
ceo
wait
adopt
bawl
move
slate
spincom
provid
belinski
winner
cupcak
grouptext
subscript
australian
apart
rebeltv
aptur
know
overflow
patientslikemecom
complex
sxswlink
web3
wrapup
master
quot
ca
noon
chromeo
commun
block
catherin
symphoni
onsit
sw
fanquot
schedul
brunch
accelerat
agileag
ellen
geekspringbreak
2wk
googletv
headlong
monday
10pm
brave
sprint
hmmmm
lol
open
pray
chimp
collect
aristotl
test
design
att
intrvw
showcas
moonbot
packard
leisureplanningrest
900
ps
plan
battledeck
touchquot
quotdop
stock
bing
idl
reaction
magnifi
fame
roam
yet
andor


Spell Correction

In [18]:
bow_1 = df['tweet'].apply(lambda x:word_tokenize(str(x)))
print(bow_1)

0       [#, sxswnui, #, sxsw, #, apple, defining, lang...
1       [Learning, ab, Google, doodles, !, All, doodle...
2       [one, of, the, most, in-your-face, ex, ., of, ...
3       [This, iPhone, #, SXSW, app, would, b, pretty,...
4       [Line, outside, the, Apple, store, in, Austin,...
5       [#, technews, One, lone, dude, awaits, iPad, 2...
6       [SXSW, Tips, ,, Prince, ,, NPR, Videos, ,, Toy...
7       [NU, user, RT, @, mention, New, #, UberSocial,...
8       [Free, #, SXSW, sampler, on, iTunes, {, link, ...
9       [I, think, I, might, go, all, weekend, without...
10      [RT, @, mention, Official, #, SXSW, App, Û÷SX...
11      [It, 's, official, !, I, 'm, buying, an, iPad,...
12      [They, 're, giving, away, iPad, 2, 's, ,, x, b...
13      [RT, @, mention, We, 're, officially, at, #, S...
14      [#, Companies, to, watch, ,, from, the, #, SXS...
15      [RT, @, mention, Google, Marissa, Mayer, ,, fu...
16      [DL, the, #, Calyp, App, to, get, into, #, Cal...
17      [Well,

In [19]:
bow_2 = []
for i in bow_1:
    bow_2.append(i)
bow_2

[['#',
  'sxswnui',
  '#',
  'sxsw',
  '#',
  'apple',
  'defining',
  'language',
  'of',
  'touch',
  'with',
  'different',
  'dialects',
  'becoming',
  'smaller'],
 ['Learning',
  'ab',
  'Google',
  'doodles',
  '!',
  'All',
  'doodles',
  'should',
  'be',
  'light',
  ',',
  'funny',
  '&',
  'amp',
  ';',
  'innovative',
  ',',
  'with',
  'exceptions',
  'for',
  'significant',
  'occasions',
  '.',
  '#',
  'GoogleDoodle',
  '#',
  'sxsw'],
 ['one',
  'of',
  'the',
  'most',
  'in-your-face',
  'ex',
  '.',
  'of',
  'stealing',
  'the',
  'show',
  'in',
  'yrs',
  'RT',
  '@',
  'mention',
  '&',
  'quot',
  ';',
  'At',
  '#',
  'SXSW',
  ',',
  'Apple',
  'schools',
  'the',
  'mkt',
  'experts',
  '&',
  'quot',
  ';',
  '{',
  'link',
  '}'],
 ['This',
  'iPhone',
  '#',
  'SXSW',
  'app',
  'would',
  'b',
  'pretty',
  'awesome',
  'if',
  'it',
  'did',
  "n't",
  'crash',
  'every',
  '10mins',
  'during',
  'extended',
  'browsing',
  '.',
  '#',
  'Fuckit',
  '

In [20]:
bow_3 = []
def reemovNestings(l): 
    for i in l: 
        if type(i) == list: 
            reemovNestings(i) 
        else: 
            bow_3.append(i)
reemovNestings(bow_2)
bow_3

['#',
 'sxswnui',
 '#',
 'sxsw',
 '#',
 'apple',
 'defining',
 'language',
 'of',
 'touch',
 'with',
 'different',
 'dialects',
 'becoming',
 'smaller',
 'Learning',
 'ab',
 'Google',
 'doodles',
 '!',
 'All',
 'doodles',
 'should',
 'be',
 'light',
 ',',
 'funny',
 '&',
 'amp',
 ';',
 'innovative',
 ',',
 'with',
 'exceptions',
 'for',
 'significant',
 'occasions',
 '.',
 '#',
 'GoogleDoodle',
 '#',
 'sxsw',
 'one',
 'of',
 'the',
 'most',
 'in-your-face',
 'ex',
 '.',
 'of',
 'stealing',
 'the',
 'show',
 'in',
 'yrs',
 'RT',
 '@',
 'mention',
 '&',
 'quot',
 ';',
 'At',
 '#',
 'SXSW',
 ',',
 'Apple',
 'schools',
 'the',
 'mkt',
 'experts',
 '&',
 'quot',
 ';',
 '{',
 'link',
 '}',
 'This',
 'iPhone',
 '#',
 'SXSW',
 'app',
 'would',
 'b',
 'pretty',
 'awesome',
 'if',
 'it',
 'did',
 "n't",
 'crash',
 'every',
 '10mins',
 'during',
 'extended',
 'browsing',
 '.',
 '#',
 'Fuckit',
 '#',
 'Illmakeitwork',
 'Line',
 'outside',
 'the',
 'Apple',
 'store',
 'in',
 'Austin',
 'waiting',
 

In [21]:
# Bow_final has all unique words of tweets ready to be converted into features for ML Model.

bow_final = list(set(bow_3))
bow_final

['inferior',
 'quick',
 '-hope',
 'mm',
 'Deck',
 'ireport',
 'AugmentedReality',
 'virginity',
 'level',
 'pics',
 'Interview',
 'Karateka',
 'client',
 'turns',
 'Vacation',
 'rescue',
 'Rest',
 'Beta',
 'than',
 'Hoping',
 'heal',
 'ur',
 'stribpol',
 'Bing',
 'Added',
 'every',
 'Compilation',
 'Script',
 'Adwords',
 'Space',
 'notes',
 'Fluffy',
 'broadcastr',
 'proving',
 '141st',
 'Connect',
 'goal',
 '\x89ÛÏContextual',
 'official',
 'Sugar',
 'Admission',
 'amigos',
 'WUT.',
 'free',
 'conferences',
 'ubiquity',
 'screen-scrape',
 'Doesnt',
 'PLAYSXSW',
 'Underwire',
 '//t.co/kfN3f5Q',
 'highly',
 'flannel',
 'loyalty',
 '60+',
 'PubSubHubbub',
 'stylish',
 'above',
 'distorted',
 'WUT',
 'out',
 'iminfanboycentral',
 'geogames',
 'appointments',
 'trumping',
 'mae',
 'non-iPhone',
 'keynote',
 'p.r',
 'apply-',
 'thick',
 'GWT',
 'freecreditscore',
 'Fascinating',
 'Fill',
 'eurorscg',
 'maybe',
 'horizon',
 'Gotto',
 'Taking',
 'sipping',
 'acquisition',
 'habit',
 'Email',


In [25]:
len(bow_final)

11938

In [22]:
without_special_char=list(filter(lambda x:x, map(lambda x:re.sub(r'[^A-Za-z]', '', x), bow_final)))

In [23]:
without_special_char

['inferior',
 'quick',
 'hope',
 'mm',
 'Deck',
 'ireport',
 'AugmentedReality',
 'virginity',
 'level',
 'pics',
 'Interview',
 'Karateka',
 'client',
 'turns',
 'Vacation',
 'rescue',
 'Rest',
 'Beta',
 'than',
 'Hoping',
 'heal',
 'ur',
 'stribpol',
 'Bing',
 'Added',
 'every',
 'Compilation',
 'Script',
 'Adwords',
 'Space',
 'notes',
 'Fluffy',
 'broadcastr',
 'proving',
 'st',
 'Connect',
 'goal',
 'Contextual',
 'official',
 'Sugar',
 'Admission',
 'amigos',
 'WUT',
 'free',
 'conferences',
 'ubiquity',
 'screenscrape',
 'Doesnt',
 'PLAYSXSW',
 'Underwire',
 'tcokfNfQ',
 'highly',
 'flannel',
 'loyalty',
 'PubSubHubbub',
 'stylish',
 'above',
 'distorted',
 'WUT',
 'out',
 'iminfanboycentral',
 'geogames',
 'appointments',
 'trumping',
 'mae',
 'noniPhone',
 'keynote',
 'pr',
 'apply',
 'thick',
 'GWT',
 'freecreditscore',
 'Fascinating',
 'Fill',
 'eurorscg',
 'maybe',
 'horizon',
 'Gotto',
 'Taking',
 'sipping',
 'acquisition',
 'habit',
 'Email',
 'incorrect',
 'wacky',
 'TRO

Spell check

In [24]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(without_special_char)

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

beforetwitter
{'beforetwitter'}
guys
{'guys'}
weeks
{'seeks', 'gees', 'greeks', 'meeks', 'reeks', 'weeks', 'leeks', 'geek'}
hashing
{'hashing'}
hmi
{'hmi', 'hdm', 'hdi'}
pop-up
{'pop-up'}
report
{'report'}
alu
{'acl', 'acls', 'alu', 'actu'}
newandroid
{'newandroid'}
bodyaches
{'bodyaches'}
benmcgraw
{'benmcgraw'}
possibly
{'wobbly', 'polly', 'pebbly', 'poly', 'portly', 'nobly', 'posy', 'doubly', 'mostly', 'possibly', 'mosely', 'costly', 'bobbly', 'cosby', 'cosily', 'polsby', 'poorly', 'posey'}
bus
{'cous', 'c-bus', 'cpus', 'cbs', 'crus', 'bus', 'cubs', 'mbus', 'sbus'}
emma
{'emma', 'hama', 'pmma', 'umma', 'hmm', 'amma', 'hmmm'}
eatshopaustinapp
{'eatshopaustinapp'}
recs
{'recor', 'recs', 'repos', 'cecos', 'pecos'}
shhh
{'ahhhh', 'ohhhh', 'shhh'}
loch
{'loch', 'nock', 'nosh', 'noah', 'goch', 'nach', 'roch', 'coch', 'koch', 'hoch', 'nch', 'och', 'noth', 'enoch', 'notch', 'anoch', 'foch'}
gonna
{'goon', 'gonna', 'goons', 'poona', 'gooda'}
years
{'aaaaa', 'year', 'meana', 'seaga', 'yaga', 

webdoccom
{'webdoccom'}
payroll
{'payroll', 'mayall', 'pascall'}
artikelen
{'artikelen'}
hamsandwich
{'hamsandwich'}
relocation
{'relocation', 're-location'}
wiredcom
{'wiredcom'}
travelled
{'travelled'}
assistivetech
{'assistivetech'}
housecoat
{'housecoat'}
assure
{'assure'}
flicking
{'tricking', 'frisking', 'pricking', 'flicking'}
turnstones
{'turnstones'}
pds
{'pcx', 'pdf', 'pdr', 'pox', 'px', 'pbx', 'pyx', 'pd', 'pdc', 'pdt', 'dpx', 'pex', 'dx', 'pdg', 'pda', 'pdp', 'pax', 'pdb', 'pdq', 'pds', 'pdm'}
eatdrinktweet
{'eatdrinktweet'}
raidsxsw
{'raidsxsw'}
context
{'context'}
dailygrape
{'dailygrape'}
getjarsxsw
{'getjarsxsw'}
glowinthedark
{'glowinthedark'}
attis
{'battys', 'attis'}
tobeannounced
{'tobeannounced'}
elitebook
{'elitebook'}
bantu
{'bunte', 'bantu', 'runtu', 'bunty'}
dumbanddumber
{'dumbanddumber'}
kasbah
{'kasbah'}
sxswgood
{'sxswgood'}
teamandroid
{'teamandroid'}
xperiaplay
{'xperiaplay'}
mediators
{'diapers', 'mediators', 'mediates'}
outcast
{'outcast', 'outlast'}
ju

digibiz
{'digibiz'}
googleaclueff
{'googleaclueff'}
netflix
{'netflix'}
att
{'atn', 'aten', 'att.', 'atta', 'att', 'atte'}
kip
{'kip'}
ttp
{'ttp'}
carnmoney
{'carnmoney'}
was
{'whs', 'wfs', 'wis', 'wus', 'wcs', 'gks', 'wkts', 'was', "w's", 'wes', 'kws', 'ks', 'wvs', 'ws'}
query
{'query'}
spiderhouse
{'spiderhouse'}
googlesponsored
{'googlesponsored'}
schwa
{'schwab', 'schwa'}
bell
{'bulli', 'bello', 'bells', 'gelli', 'bell', 'belly', 'bella', 'belle', 'bellis'}
dead
{'deaden', 'daddy', 'leaded', 'dreaded', 'headed', 'dddd', 'dadda', 'beaded', 'dead', 'deadly', 'dadd'}
livetapp
{'livetapp'}
issue
{'esau', 'esme', 'esq', 'duque', 'mosque', 'basque', 'bisque', 'nque', 'masque', 'esq.', 'que', 'issue', 'ensue', 'pique', 'auque', 'rescue', 'roque', 'esquire', 'sequel', 'fescue', 'este', 'risque', 'esse', 'epoque', 'sue'}
people
{'people'}
tsotchke
{'tsotchke'}
musicviz
{'musicviz'}
wow
{'fwwg', 'ecwp', 'ewe', 'sw1w', 'ebbw', 'e.w.', 'ewen', 'wow', 'nuww', 'ewos', 'aww', 'wwf', 'ewan', 'ew',

{'doe', 'darne', 'roe', 'dirge', 'diro', 'dire', 'dafoe', 'dare', 'wroe', 'dore', 'dro', 'dere', 'drop', 'faroe', 'defoe', 'dorje', 'joe', 'darke', 'duroc', 'durie', 'drone', 'drove', 'dron', 'daro'}
categorized
{'categorized'}
rebels
{'rebels', 'rebel'}
start
{'strabo', 'sarfu', 'stab', 'stars', 'sarb', 'tarby', 'stark', 'starve', 'starch', 'stare', 'starry', 'starr', 'stared', 'startup', 'searby', 'starbuck', 'star', 'starky', 'starts', 'start', 'saru', 'stares', 'stabs', 'starke'}
webbrowsing
{'webbrowsing'}
goodies
{'goodies'}
frankeninterface
{'frankeninterface'}
win-win
{'win-win'}
kiiiiiilling
{'kiiiiiilling'}
celebrant
{'celebrant'}
makers
{'bakery', 'makers', 'maker'}
notsomuch
{'notsomuch'}
reform
{'deform', 'reform'}
ipadformatted
{'ipadformatted'}
booth
{'hookah', 'booth', 'loofah'}
sucks
{'sucks'}
moonlit
{'moonlit'}
freshbooks
{'freshbooks'}
marshall
{'mashad', 'mahal', 'marshal', 'mascall', 'washable', 'masham', 'masha', 'marshall'}
facebookkonkurrent
{'facebookkonkurren

{'showered'}
dig
{'dii', 'dili', 'diii', 'tigi', 'didi', 'gigi', 'digs', 'digit', 'divi', 'dini', 'dig'}
specify
{'sportif', 'notify', 'specify', 'spotty'}
mc
{'jac', 'mmc', 'fmc', 'bmc', 'jmp', 'mc', 'pmc', 'lmc', 'jmu', 'jcc', 'imc', 'jmb', 'emc', 'gmc', 'tmc', 'jvc', 'hmc', 'smc', 'cmc', 'wmc', 'jc', 'jma', 'jxc', 'rmc', 'vmc', 'jm'}
webb
{'webb'}
crazy
{'crazy'}
aha
{'ha-ha', 'haya', 'hama', 'hahn', 'naha', 'hana', 'aha', 'maha', 'hah', 'taha', 'hata'}
head-down
{'head-down'}
self-respecting
{'self-respecting'}
sure
{'sur', 'shrug', 'guruji', 'sirri', 'uri', 'strut', 'syrup', 'scrub', 'surat', 'suum', 'turpi', 'sura', 'yuri', 'suu', 'susu', 'sushi', 'puri', 'sueur', 'scrum', 'spru', 'gurus', 'nuri', 'fukui', 'shui', 'shrub', 'uruk', 'samui', 'sri', 'durum', 'sure', 'sulci', 'serum', 'burun', 'sarum', 'suki', 'surer', 'murli', 'rui', 'sari', 'guru', 'suzuki', 'burgi', 'seru', 'surf', 'seri', 'murti', 'sarup', 'saru', 'sufi', 'surya', 'surly', 'sulu', 'surge', 'sui', 'aurum', 'strum'

...

winderman
{'dangerman', 'winderman'}
peacetime
{'peacetime', 'spacetime'}
panellist
{'panellist'}
creative
{'creative'}
since
{'lanced', 'sunbed', 'winced', 'sliced', 'singed', 'sync', 'senfed', 'danced', 'unced', 'synne', 'spiced', 'syed', 'syne', 'spaced', 'minced', 'since', 'sensed', 'synod', 'syncro', 'lynched', 'sanded', 'sinned', 'synch', 'stanced', 'syce', 'fenced'}
played
{'fayed', 'played', 'layer', 'bayed', 'laye', 'payed', 'laced', 'flayed', 'sayed', 'zayed', 'lazed'}
convert
{'convene', 'conform', 'colmore', 'condors', 'concord', 'concorde', 'connote', 'condor', 'converge', 'convoy', 'condone', 'connor', 'converse', 'console', 'contort', 'canmore', 'conor', 'colore', 'convert', 'consort', 'connors', 'conjure', 'corpore', 'convoys', 'contre', 'convoked'}
friendsoffer
{'friendsoffer'}
lowest
{'loveitt', 'lowest', 'lovesick', 'lovett', 'loves', 'lossit'}
pidal
{'pidal'}
player
{'player'}
trickling
{'brickley', 'trickling', 'prickling', 'blickling', 'nicklin', 'brickie'}
dead
{

sideshows
{'sideshows'}
sell
{'axel', 'syl', 'dfl', 'sal', 'sl', 'spl', 'sfv', 'ifl', 'sql', 'sail', 'xf4', 'sohl', 'saf', 'vxl', 'sel', 'sdf', 'fl', 'sfa', 'sufi', 'sfp', 'scfa', 'xf3', 'sul', 'senfl', 'rfl', 'safi', 'vxfs', 'sifu', 'shel', 'ffl', 'safe', 'xl', 'soil', 'pfl', 'soul', 'wfl', 'swf', "s'il", 'suhl', 'sx', 'seal', 'axil', 'snl', 'srf', 'xgl', 'scf', 'npfl', 'shl', 'sfc', 'soll', 'sf', 'sfo', 'sfi', 'bnfl', 'ssl', 'safa', 'nfl', 'afl', 'sef', 'sfx', 'axl', 'xf6', 'sell', 's/l', 'snf', 'sift', 'skil', 'slf', 'sofa', 'sol', 'spf', 'self', 'dxf', 'sml', 'skol', 'sill', 'xfi', 'slfp', 'sgml', 'xf5', 'scfw', 'saul', 'xil', 'efl', 'xf7', 'soft', 'scfv', 'xcl'}
fbi
{'fyt', 'fii', 'yi', 'fym', 'fy', 'fei', 'fyn', 'fmi', 'fi', 'kyi', 'fdi', 'fai', 'fbi', 'fki', 'fpi', 'f.i', 'fri'}
donegal
{'donegal', 'negri', 'onetti', 'omega', 'onegin', 'lengai', 'nogai', 'ongar', 'donegan'}
streetviewlike
{'streetviewlike'}
penney
{'penney'}
contactless
{'contactless'}
marisa
{'marisa'}
mention


space
{'space', 'espace'}
tml
{'tml'}
icl
{'ipal', 'icas', 'icao', 'ica', 'icl', 'cal', 'kcal'}
authenticated
{'authenticate', 'authenticated', 'authentication'}
marketshare
{'marketshare'}
clumps
{'humps', 'chums', 'thumps', 'clumps', 'champs', 'chump', 'crumps', 'chimps'}
lifelinetotheworld
{'lifelinetotheworld'}
apl
{'apl'}
echo
{'gecko', 'echo', 'eco', 'esko', 'eck'}
germanversion
{'germanversion'}
btw
{'btw'}
foxton
{'foxton'}
catalogued
{'catalogued'}
future
{'futures', 'future'}
dna
{'jana', 'enya', 'nonya', 'janna', 'kenya', 'tanya', 'jonna', 'jaya', 'rdna', 'konya', 'zanya', 'minya', 'jna', 'dna', 'daya', 'anya', 'cdna', 'sonya', 'jenna', 'junta', 'jena', 'edna'}
microfilm
{'microfilm', 'microfit', 'micros'}
iamagameshater
{'iamagameshater'}
mishmash
{'mishmash'}
conchis
{'cookie', 'cochin', 'conchis', 'coolie', 'cloche'}
pods
{'pods', 'ipos'}
pressurise
{'pressurise'}
prey
{'prey', 'pre-', 'prem', 'pre', 'pres', 'grev', 'prv', 'prep', 'rev', 'trev'}
peemekylj
{'peemekylj'}
mi

reviewrecommendations
{'reviewrecommendations'}
nfcrfid
{'nfcrfid'}
early
{'nearly', 'fairly', 'charly', 'pearly', 'gnarly', 'yearly', 'dearly', 'early', 'carly'}
blackberry
{'blackberry'}
slammers
{'slammers'}
moment
{'momently', 'momentum', 'momento', 'moment', 'moments', 'momenta'}
imrie
{'imrie'}
bitlygvlrin
{'bitlygvlrin'}
great
{'great'}
wifionly
{'wifionly'}
searchengineland
{'searchengineland'}
hold
{'hold'}
amplifiedlife
{'amplifiedlife'}
cnnsxsw
{'cnnsxsw'}
idontbelieve
{'idontbelieve'}
stoned
{'stoned', 'intoned'}
press
{'press', 'pres.', 'prest', 'peso', 'pres', 'presto'}
popchicks
{'popchicks'}
lynching
{'lynching'}
prober
{'prober', 'proba'}
epictweets
{'epictweets'}
ignitesxsw
{'ignitesxsw'}
body
{'booth', 'bora', 'boy', 'boob', 'buoy', 'botha', 'nonya', 'booz', 'biya', 'bloye', 'boys', 'boyd', 'boots', 'bony', 'booze', 'boyos', 'bocca', 'boyo', 'booed', 'bona', 'boyz', 'buyoya', 'bogota', 'boop', 'konya', 'boos', 'boma', 'mboya', 'boozy', 'poona', 'moya', 'boa', 'book',

five-strong
{'five-strong'}
quote
{'quoc', 'qdot', 'quote', 'quit', 'quoth', 'quota', 'quoit', 'quo', 'quod'}
flypast
{'flypast'}
boobs
{'boobs', 'nobbs', 'nooks', 'nobs'}
lava
{'kava', 'sava', 'iva', 'iaea', 'lava', 'itva', 'cava', 'java', 'iata', 'iawa', 'ava', 'nava'}
scoring
{'scoring', 'sloping', 'scooping', 'coping'}
batterykiller
{'batterykiller'}
bishop
{'bimbo', 'bebop', 'bishop', 'bilbo', 'bitmap', 'bimbos', 'bitow'}
modem
{'modem', 'idem'}
instaprint
{'instaprint'}
ceokidschat
{'ceokidschat'}
desmiente
{'desmiente'}
pan-american
{'pan-american'}
snub
{'stupor', 'lubor', 'suor', 'snub', 'sobor', 'snubs'}
soundcloud
{'soundcloud'}
authorized
{'authorized'}
journalism
{'journalism'}
marissameyer
{'marissameyer'}
mitharvard
{'mitharvard'}
sponsor
{'sponsor'}
aww
{'aww'}
tariq
{'tariq'}
goal
{'gca', 'gal', 'goal', 'cal', 'kcal'}
checkingout
{'checkingout'}
popupstore
{'popupstore'}
conservebatterylife
{'conservebatterylife'}
youplease
{'youplease'}
googleme
{'googleme'}
yourmom
{

In [36]:
pwd

'C:\\Users\\praks\\Desktop\\sentiment hackathon\\data'